In [1]:
import tensorflow as tf
import datetime

# Load the TensorBoard notebook extension
%load_ext tensorboard

!echo %CD%
!mkdir .\logs
!mkdir .\logs\lab-10-3-mnist_nn_xavier-tf2
!rmdir /q /s .\logs\lab-10-3-mnist_nn_xavier-tf2

log_dir = "./logs/lab-10-3-mnist_nn_xavier-tf2/"

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = log_dir + current_time + '/train'
test_log_dir = log_dir + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)


C:\Users\greenblue\Work\study\machineLearning\DeepLearningZeroToAll


하위 디렉터리 또는 파일 .\logs이(가) 이미 있습니다.


In [ ]:
# Lab 10 MNIST and Xavier
from functools import partial as bind
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
assert(tf.__version__.find('2') == 0)
tf.random.set_seed(777)  # for reproducibility

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = tf.reshape(x_train, [-1, 28*28])
x_train = tf.cast(x_train, tf.float32)
y_train = tf.one_hot(y_train, 10)
y_train = tf.cast(y_train, tf.float32)
x_train_len = len(x_train)
y_train_len = len(y_train)
# print(x_train_len, y_train_len)
# print(x_train, y_train)

x_test = tf.reshape(x_test, [-1, 28*28])
x_test = tf.cast(x_test, tf.float32)
y_test = tf.one_hot(y_test, 10)
x_test_len = len(x_test)
y_test_len = len(y_test)
# print(x_test_len, y_test_len)
# print(x_test, y_test)

learning_rate = 0.001
nb_classes = 10
batch_size = 100
training_epochs = 15
num_iterations = int(x_train_len / batch_size)

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[784, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random.normal([256]))
L1 = lambda X: tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random.normal([256]))
L2 = lambda X: tf.nn.relu(tf.matmul(L1(X), W2) + b2)

W3 = tf.get_variable("W3", shape=[256, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random.normal([10]))
hypothesis = lambda X: tf.matmul(L2(X), W3) + b3

# define cost/loss & optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

@tf.function
def cost(X, Y):
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis(X),
                                                labels=Y)
    )

@tf.function
def correct_prediction(X, Y):
    return tf.equal(tf.argmax(hypothesis(X), axis=1), tf.argmax(Y, axis=1))

@tf.function
def accuracy(X, Y):
    return tf.reduce_mean(tf.cast(correct_prediction(X, Y), tf.float32))

# train my model
for epoch in range(training_epochs):
    avg_cost = 0

    for i in range(num_iterations):
        pos = i * batch_size
        batch_xs = x_train[pos: pos + batch_size, :]
        batch_ys = y_train[pos: pos + batch_size, :]
#         print(batch_xs)
#         print(batch_ys)
        
        cost_val = cost(batch_xs, batch_ys)
        avg_cost += cost_val / num_iterations

        # Minimize
        optimizer.minimize(bind(cost, batch_xs, batch_ys), var_list=[W1, b1, W2, b2, W3, b3])

        step = epoch * num_iterations + i
        if step % 100 == 0:
            with train_summary_writer.as_default():
                tf.summary.scalar('loss', cost_val, step=step)
                tf.summary.scalar('accuracy', accuracy(batch_xs, batch_ys), step=step)
        
    tf.print("Epoch: {:04d}, Cost: {:.9f}".format(epoch + 1, avg_cost))

tf.print("Learning finished")

# Test model and check accuracy
tf.print(
    "Accuracy: ",
    accuracy(x_test, y_test)
)

# Get one and predict
r = random.randint(0, y_test_len - 1)
tf.print(
    "Label: ", tf.argmax(y_test[r : r + 1], 1)
)
tf.print(
    "Prediction: ",
    tf.argmax(hypothesis(x_test[r : r + 1]), 1),
)

plt.imshow(
    tf.reshape(x_test[r : r + 1], [28, 28]),
    cmap="Greys",
    interpolation="nearest",
)
plt.show()

'''
Epoch: 0001 cost = 0.301498963
Epoch: 0002 cost = 0.107252513
Epoch: 0003 cost = 0.064888892
Epoch: 0004 cost = 0.044463030
Epoch: 0005 cost = 0.029951642
Epoch: 0006 cost = 0.020663404
Epoch: 0007 cost = 0.015853033
Epoch: 0008 cost = 0.011764387
Epoch: 0009 cost = 0.008598264
Epoch: 0010 cost = 0.007383116
Epoch: 0011 cost = 0.006839140
Epoch: 0012 cost = 0.004672963
Epoch: 0013 cost = 0.003979437
Epoch: 0014 cost = 0.002714260
Epoch: 0015 cost = 0.004707661
Learning Finished!
Accuracy: 0.9783
'''
